# 1. Word2Vec

### 1.1 SkipGram模型的推导

两种训练模式：

1. **SkipGram:** 根据中心词预测上下文词（<font color='#c63c26'>效果更好，因为同样的文本可以得到更多的训练样本</font>）

2. **CBOW:** 根据上下文词预测中心词

下面主要讲SkipGram

对于一句话: 
`We are working on NLP project, it is interesting`

SkipGram的目标是Maximize下面的目标函数:

$$
\begin{array}{l}
\text{Maximize} ~ P(are \mid we)\cdot P(working \mid are) \cdots P(it \mid interesting)\cdot P(is \mid interesting) \\
\text{Maximize} ~ \prod_{w \in Text} \prod_{c \in Context(w)} P(c \mid w;\theta) \\
\text{Maximize} ~ \sum_{w \in Text} \sum_{c \in Context(w)} \log P(c \mid w;\theta)
\end{array}
$$

<font color=blue>问题的关键是如何表示$P(c \mid w;\theta)$</font>

<img src='attachment/19. word2vec 词向量矩阵.png' style='zoom:50%'/>

$$
\color{red}{P(c \mid w;\theta)} = \color{red}{\frac{e^{u_{c} ~ \cdot v_{w}}}{\sum_{c' \in vocab} e^{u_{c^{\prime}} ~ \cdot v_{w}}}}
$$

最终的优化问题成为: 

$$
\arg \max _{u,v} \sum_{w \in Text} \sum_{c \in Context(w)} \log \frac{e^{u_{c} ~ \cdot v_{w}}}{\sum_{c' \in vocab} e^{u_{c^{\prime}} ~ \cdot v_{w}}}
$$

#### 示例
设预料为： 

`"今天 是 周六 上 NLP 是"`， $\text{dim}=3, ~ \text{window_size}=1$，则优化问题为: 

$$
\begin{align}
\text{Maximize} &~ P(是|今天)P(今天|是)P(周六|是)P(上|周六)P(是|周六)\\
& P(周六|上)P(NLP|上)P(上|NLP)P(是|NLP)P(是|NLP) \\ 
\end{align}
$$

$$
\begin{align}
\text{Maximize} &~ \frac{e^{u_{是}\cdot v_{今天}}}{\sum_{c^{\prime}} e^{u_{c^{~\prime}}\cdot v_{今天}}} \cdot \frac{e^{u_{今天}\cdot v_{是}}}{\sum_{c^{\prime}} e^{u_{c^{~\prime}}\cdot v_{是}}}\cdots \frac{e^{u_{是}\cdot v_{NLP}}}{\sum_{c^{\prime}} e^{u_{c^{~\prime}}\cdot v_{NLP}}}\\
\end{align}
$$

因此，SkipGram模型参数$u,v$的求解就是Maximize下面的问题: 

$$
\begin{align}
\text{Maximize} &~ \sum_{w \in Text} \sum_{c \in \text{context}(w)}  \log \frac{e^{u_{c}~ \cdot v_{w}}}{\sum_{c'}e^{u_{e^{\prime}}\cdot v_{w}}} \\
&= \sum_{w \in Text} \sum_{c \in \text{context}(w)} u_{c} \cdot v_{w} - \color{red}{ \log \left(\sum_{c^{\prime}} e^{u_{e^{\prime}}\cdot v_{w}} \right)} \ \color{blue}{O(|V|)，不好求} \tag{1.1}
\end{align}
$$ 

### 1.2 Another Formulation

SkipGram的原始形式中存在$\color{red}{\log \sum}$形式，Intractable。

对于一个文本：
$\color{#9932CC}{S = w_{1}~w_{2}~w_{3}~w_{4}~w_{5}~w_{6}}$.

在原始形式中，我们知道token与token之间的位置信息，然后最大化似然函数。

在新的视角中，可以通过corpus训练模型判断token与token之间是否是上下文词关系从而学得Embedding，比如根据 $\color{#9932CC}{S}$ 可以得到两两词之间的关系的概率判别式：

$$
P(D=1 \mid w_{1}, w_{2}) \\
P(D=0 \mid w_{1}, w_{2}) \\
P(D=1 \mid w_{1}, w_{3}) \\
P(D=0 \mid w_{1}, w_{3}) \\
\cdots \\
P(D=1 \mid w_{5}, w_{6}) \\
P(D=0 \mid w_{5}, w_{6}) \\
$$

利用Sigmoid函数来表示概率判别式: 

$$
\begin{align}
P(D=1 \mid w_{i}, w_{j}) &= \frac{1}{1 + \exp (-u_{w_{i}}\cdot v_{w_{j}})} = \sigma(u_{w_{i}}\cdot v_{w_{j}}) \\
P(D=0 \mid w_{i}, w_{j}) &= 1 - P(D=1 \mid w_{i}, w_{j})
\end{align}
$$

#### 示例

window_size = 1 

`S = "Today's weather is great"`

`vocab = [Today's, weather, is, great]`

正样本集合: `D={(Today's, weather), (weather, Today's), (weather, is), (is, weather), (is, great), (great, is)}`

负样本集合: `D'={(Today's, is), (Today's, great), (weather, great), (is, Today's), (great, Today's), (great, weather)}`

则优化问题为极大化如下似然函数：

$$
\begin{align}
\text{Maximize} &~ P(D=1 \mid Today's,~ weather) \cdots P(D=1 \mid great,~ is) \\
& \cdot P(D=0 \mid Today's,~ is) \cdots P(D=0 \mid great,~ weather)
\end{align}
$$

一般地，可以用更通用的形式表示为：

$$
\begin{align}
\text{Maximize} &~ \prod_{(w,c)\in D} P\left(D=1 \mid w,c;\theta\right) \cdot \prod_{(w,c)\in \tilde{D}}P\left(D=0 \mid w,c;\theta\right) \\
\\
&= \prod_{(w,c)\in D} P\left(D=1 \mid w,c;\theta\right) \cdot \prod_{(w,c)\in \tilde{D}} \left[1 -  P\left(D=0 \mid w,c;\theta\right) \right] \\
\\
&= \sum_{(w,c)\in D} \log \sigma \left(u_{w}\cdot v_{c} \right) + \sum_{\color{red}{(w,c)\in \tilde{D}}} \log \sigma \left(-u_{w}\cdot v_{c} \right) \tag{1.2}
\end{align}
$$

可以看到，上式是不含$\color{red}{\log \sum}$的。

### 1.3 负采样

在(1.2)式中，假设 $|vocab|=10^{5}$，则$|D| + |\tilde{D}| \approx 10^{10}$，且$|D| \ll |\tilde{D}|$

所谓负采样，就是从每个正样本中，随机采样若干个负样本，进行训练：

$$
\begin{align}
\text{Maximize} &~ \sum_{(w,c)\in D} \left[\underset{l(\theta)}{ \underbrace{ \log \sigma(u_{w}, v_{c}) + \sum_{c^{\prime} \in Negative(w)} \log \sigma (- u_{w} \cdot v_{c^{\prime}}) } }\right]  \\
\frac{\partial l(\theta)}{\partial u_{w}} &= \frac{\sigma (- u_{w} \cdot v_{c^{\prime}})[1-\sigma (- u_{w} \cdot v_{c^{\prime}})]\cdot v_{c}}{\sigma (- u_{w} \cdot v_{c^{\prime}})} + \sum_{c^{\prime} \in Negative(w)} \frac{\sigma (- u_{w} \cdot v_{c^{\prime}}) [1-\sigma (- u_{w} \cdot v_{c^{\prime}})]\cdot (-v_{c^{\prime}})}{\sigma (- u_{w} \cdot v_{c^{\prime}})} \\
\frac{\partial l(\theta)}{\partial v_{c}} &= \\
\frac{\partial l(\theta)}{\partial v_{c^{\prime}}} &=
\end{align}
$$ 

### 1.4 MF（Matrix Factorization，矩阵分解）

MF是一种全局的方法

设预料为：

```
I back my car
I back the car
Back my car
```

共现矩阵:

<img src='attachment/19. MF共现矩阵.png' style='zoom:50%' /> 

通过矩阵分解，可得到词向量

$$
A = C^{\mathrm{T}} V
$$

MF的<font color=red>缺点</font>：
- 无法很好处理新词；
- 矩阵分解计算量大

<font color=red>GloVe是整合MF和SkipGram优点而成的词向量训练方法。</font>

### 1.5 Gaussian Embedding

SkipGram, CBOW等Embedding有一个缺点，就是Embedding的学习没有注意到不确定性，<font color=blue>对于高频词学习较好，但是低频词学习较差。</font>

Gaussian Embedding在学得embedding的同时，学出置信度。比如$E_{I} \sim N(u_{I}, \Sigma_{I}),~ E_{good} \sim N(u_{good}, \Sigma_{good})$

# 2. 语言模型LM

### 2.1 Language Model

语言模型：`从语法上判断一句话是否通顺`。

用数学语言表示为：

$$
P(\text{Today is sunday}) > P(\text{Today sunday is})
$$

### 2.2 Makov Assumption

### 2.3 Unigram、Bigram、N-gram

- Unigram:
$$
P(\text{Today is sunday}) = P(\text{Today})P(\text{is})P(\text{sunday})
$$

- Bigram
$$
P(\text{Today is sunday}) = P(\text{Today})P(\text{is} \mid \text{Today})P(\text{sunday} \mid \text{is})
$$

### 2.4 语言模型的评估

理想情况下，可以选定一个特定的任务，比如拼写纠错，测试两个语言模型的表现。

**Perplexity**

$$
\text{Perplexity} = 2^{-x} \quad x:\text{average log likelihood}
$$

比如：`S="I'm good at studying"`
$$
x = \frac{1}{4}\left( \log P(\text{I'm}) + \log P(\text{good}\mid \text{I'm}) + \log P(\text{at}\mid \text{good}) + \log P(\text{studying}\mid \text{at}) \right)
$$

# 3. Smoothing技术

在N-gram模型中，如果某些词的组合没有出现，它的概率为0，是不合理的，因此需要进行smoothing操作。调整MLE的概率值，把0概率调高，把高概率调低，改进模型的整体正确率。

### 3.1 Add-one smoothing（Laplace smoothing）

对Bigram,有
$$
\begin{align}
p(w_{i}\mid w_{i-1}) &= \frac{1+c(w_{i-1}w_{i})}{\sum_{w_{i}}\left[1+c(w_{i-1}w_{i})\right]}\\
&= \frac{1+c(w_{i-1}w_{i})}{|V| + \sum_{w_{i}}c(w_{i-1}w_{i})}
\end{align}
$$

假设语料库S由三个句子组成：
```
Brown read holy Bible
Mark read a text book
He read a book by David
```
用Bigram和MLE求$P(\text{Brown read a book})$
- $P(\text{Brown}\mid \text{<BOS>}) = \frac{1}{3}$
- $P(\text{read}\mid \text{Brown}) = \frac{1}{1}$
- $P(\text{a}\mid \text{read}) = \frac{1}{2}$
- $P(\text{<EOS>}\mid \text{book}) = \frac{1}{2}$

用Bigram和加一平滑法求$P(\text{Brown read a book})$
- $P(\text{Brown}\mid \text{<BOS>}) = \frac{1+1}{11+3} = \frac{2}{14}$
- $P(\text{read}\mid \text{Brown}) = \frac{1+1}{11+1} = \frac{2}{12}$
- $P(\text{a}\mid \text{read}) = \frac{1+1}{11+2} = \frac{2}{13}$
- $P(\text{<EOS>}\mid \text{book}) = \frac{1+1}{11+2} = \frac{2}{13}$

<font color=blue>加1平滑通常情况下是一种很糟糕的算法，与其他平滑方法相比显得非常差，然而我们可以把加1平滑用在其他任务中，如文本分类，或者非零计数没有那么多的情况下。</font>

### 3.1 Good-Turing 